In [2]:
pip install google.colab

ERROR: Could not find a version that satisfies the requirement google.colab (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for google.colab
Note: you may need to restart the kernel to use updated packages.


In [3]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the shared folder path in MyDrive
shared_folder_path = '/content/drive/MyDrive/MOASS'

# List the contents of the folder
try:
    files = os.listdir(shared_folder_path)
    print("Files in shared folder:", files)
except FileNotFoundError:
    print("Shared folder not found, please check the folder path.")


ModuleNotFoundError: No module named 'google.colab'

In [16]:
file_to_annotate = '/content/drive/MyDrive/MOASS/Annotate/rDDintoGME_comments_features_500.csv'

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

def annotate_from_csv(csv_file, score_column, annotator_name, annotation_column=None):
    """
    Function to annotate comments from a CSV file and save the results.

    csv_file: str - Path to the CSV file.
    score_column: str - The column with Vader scores.
    annotator_name: str - The name of the annotator.
    annotation_column: str - The column where annotations will be stored (optional).
    """
    # Generate annotation column name if not provided
    if annotation_column is None:
        annotation_column = f'annotation_{annotator_name}'

    # Load CSV data
    df = pd.read_csv(csv_file)

    # Ensure the annotation column exists in the DataFrame
    if annotation_column not in df.columns:
        df[annotation_column] = None  # Add the annotation column if it doesn't exist

    # Filter rows that have a score and no existing annotation
    unannotated_rows = df[df[score_column].notna() & df[annotation_column].isna()]

    # Check if there are any unannotated rows
    if unannotated_rows.empty:
        print("No rows left to annotate.")
        return

    # Initialize a counter to go through the unannotated rows
    row_idx = 0

    def display_row(idx):
        clear_output(wait=True)  # Clear the previous output before displaying the new one
        row = unannotated_rows.iloc[idx]

        # Display the comment for annotation
        comment_label = widgets.HTML(value=f"<b>Comment:</b> {row['comments']}")

        # Display the original Vader score
        score_label = widgets.HTML(value=f"<i>Original Vader Score: {row[score_column]}</i>")

        # Create a slider to annotate the score, initialized with the original Vader score
        slider = widgets.FloatSlider(
            value=row[score_column],  # Start the slider at the value of the original Vader score
            min=-1.0, max=1.0, step=0.01, description="Score:"
        )

        confirm_btn = widgets.Button(description="Confirm")

        # When the user confirms, the annotation is saved and the next row is displayed
        def on_confirm(b, _idx=idx, _slider=slider):
            df.at[_idx, annotation_column] = _slider.value  # Save the annotation
            df.at[_idx, 'annotator_name'] = annotator_name  # Save annotator's name
            save_df(df, csv_file)  # Save the updated DataFrame immediately after each annotation
            print(f"Annotation saved for row {_idx}: {_slider.value}")
            b.disabled = True  # Disable the button after confirmation

            # Move to the next row (if any)
            if idx + 1 < len(unannotated_rows):
                display_row(idx + 1)
            else:
                print("All annotations completed.")

        confirm_btn.on_click(on_confirm)

        # Display the widgets (comment, score, slider, and confirm button)
        display(widgets.VBox([comment_label, score_label, slider, confirm_btn]))

    # Start the annotation process from the first row
    display_row(row_idx)

def save_df(df, csv_file):
    """ Save the DataFrame after annotations """
    df.to_csv(csv_file, index=False)  # Save to the original CSV file

# Example usage of the function as a widget:
annotate_from_csv(
    csv_file=file_to_annotate,  # Path to your CSV file
    score_column='neg',    # The column with Vader scores, other options 'pos'/'neu', adjust after finishing first annotation round
    annotator_name='Your Name',     # Name of the annotator
    annotation_column=None  # The column where annotations will be stored, can be dynamic
)